In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

# Their paper used a 3 layer neural network (2 hidden layers) each
# with dimension 32 each
D_in = 128
H = 32
D_out = 1

# Define Linear NN's
# Model 1 is a 3 layer NN
model1 = nn.Sequential(
        # nn.Dropout(p=0.2), # p = probability of dropping
        nn.Linear(D_in, H), # Layer 1 Weights
        # nn.Dropout(p=0.2), # p = probability of dropping
        nn.Linear(H, H), # Layer 2 Weights
        # nn.Dropout(p=0.2), # p = probability of dropping
        nn.Linear(H, D_out)  # Layer 3 Weights
      )

# Model 2 is a 8 layer NN
model2 = nn.Sequential(
        # nn.Dropout(p=0.2), # p = probability of dropping
        nn.Linear(D_in, H), # Layer 1 Weights
        # nn.Dropout(p=0.2), 
        nn.Linear(H, H), # Layer 2 Weights
        # nn.Dropout(p=0.2),
        nn.Linear(H, H), # Layer 3 Weights
        # nn.Dropout(p=0.2),
        nn.Linear(H, H), # Layer 4 Weights
        # nn.Dropout(p=0.2),
        nn.Linear(H, H), # Layer 5 Weights
        # nn.Dropout(p=0.2),
        nn.Linear(H, H), # Layer 6 Weights
        # nn.Dropout(p=0.2),
        nn.Linear(H, H), # Layer 7 Weights
        # nn.Dropout(p=0.2),
        nn.Linear(H, D_out)  # Layer 8 Weights
      )

In [2]:
import numpy as np
# Load Data
data_path = '../data/processed/'
X = np.load(data_path + 'X.npy')
y = np.load(data_path + 'y.npy')
print("X has shape: {}\ny has shape: {}.".format(X.shape, y.shape))

X has shape: (2565, 128)
y has shape: (2565, 1).


In [3]:
import sys
sys.path.append('../src/models/')
from linear_nn import three_layer_nn, fro_loss

model = three_layer_nn()

In [4]:
X_torch = torch.from_numpy(X).float()
y_torch = torch.from_numpy(y).float()
print(model.layer_1.weight.shape)
print(model.layer_2.weight.shape)
print(model.layer_3.weight.shape)
# output = model.forward()
# output
print((y_torch.transpose(0,1) @ X_torch).shape)

torch.Size([32, 128])
torch.Size([32, 32])
torch.Size([1, 32])
torch.Size([1, 128])


In [5]:
W_1 = model.layer_1.weight
W_2 = model.layer_2.weight
W_3 = model.layer_3.weight
W = W_3 @ W_2 @ W_1

In [6]:
loss = fro_loss()
loss(W, X_torch, y_torch)

tensor(0.0173, grad_fn=<MulBackward>)